## 1.1 MyTime

Định ra class MyTime để lưu lại thời gian trong ngày. Class định ra sẽ trông như thế này:

In [35]:
class MyTime:
    def __init__(self, hrs=0, mins= 0, secs=0):
        """ Create a MyTime object initialized to hrs, mins, secs """
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs
    
    def __str__(self):
        return "({0}:{1}:{2})".format(self.hours, self.minutes, self.seconds)

In [36]:
tim1 = MyTime(11, 59, 30)
tim1

In [37]:
print(tim1)

(11:59:30)


## 1.2 Pure functions

Viết một hàm gọi là `add_time` để tổng thời gian giữa 2 objects. Cái này để chứng minh cho 2 loại function là pure function và modifier

In [38]:
def add_time(t1, t2):
    h = t1.hours + t2.hours
    m = t1.minutes + t2.minutes
    s = t1.seconds + t2.seconds
    sum_t = MyTime(h, m, s)
    return sum_t

function này được tạo bởi object MyTime và gọi là **pure function** vì nó không chỉnh sửa gì bất kì đối tượng nào được pass làm thông số.

In [39]:
current_time = MyTime(9, 14, 30)
bread_time = MyTime(3, 35, 0)
done_time = add_time(current_time, bread_time)

In [40]:
print("{}:{}:{}".format(done_time.hours, done_time.minutes, done_time.seconds))

12:49:30


Kết quả đúng nhưng ở các case khác thì kết quả sẽ không đúng. Như trong trường hợp các case có các attributes như hours, minutes, seconds mà tổng giữa t1 và t2 lớn hơn 60. Ở dưới là một phiên bản tốt hơn.

In [41]:
def add_time(t1, t2):
    h = t1.hours + t2.hours
    m = t1.minutes + t2.minutes
    s = t1.seconds + t2.seconds
    
    if s>= 60:
        s -= 60
        m += 1
    
    if m >= 60:
        m -= 60
        h +=1
        
    sum_t = MyTime(h, m, s) 
    return sum_t

Hàm này đã trở nên lớn nhưng vẫn không hiệu quả với mọi trường hợp.

## 1.3 Modifiers

Sẽ có những lúc hữu ích cho một hàm có thể chỉnh sửa một hoặc nhiều đối tượng khi đưa nó vào làm thông số. Những hàm hoạt động theo kiểu đó gọi là **modifiers**

In [42]:
def increment(t, secs):
    t.seconds += secs
    
    if t.seconds >= 60:
        t.seconds -= 60
        t.minutes +=1
    
    if t.minutes >= 60:
        t.minutes -=60
        t.hours += 1    

Dòng đầu là một phép toán đơn giản, phần còn lại là cho các trường hợp đặc biệt mà ta gặp ở trên. Vấn đề là nếu các attribute này lớn hơn rất nhiều so với 60 thì nó lại chỉ tính có 1 lần. Vậy nên phù hợp là thay `if` bằng `while`

In [43]:
def increment(t, secs):
    t.seconds += secs
    
    while t.seconds >= 60:
        t.seconds -= 60
        t.minutes +=1
    
    while t.minutes >= 60:
        t.minutes -=60
        t.hours += 1  

Hàm này đến giờ khá đúng nếu seconds không âm. Nhưng nếu thời gian vượt quá 23 giờ thì nó vẫn chưa phải giải pháp tốt.

## 1.4 Converting increment to a method

Một người lập trình định hướng đối tượng sẽ muốn sử dụng các hàm hoạt động với các object thuộc MyTime vào class MyTime

In [44]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs

    def __str__(self):
        return "{0}:{1}:{2}".format(self.hours, self.minutes, self.seconds)        
    
    def increment(self, seconds):
        self.seconds += seconds
        
        while self.seconds >= 60:
            self.seconds -=60
            self.minutes +=1
            
        while self.minutes >= 60:
            self.minutes -= 60
            self.hours += 1        

Việc chuyển đổi này khá máy móc. Ta đưa hàm này vào trong lúc định ra class MyTime. Và thay tham số đầu tiên là self. Thử với current_time như ở trên, thêm 500 giây.

In [45]:
current_time = MyTime(9, 14, 30)
current_time.increment(500)
print(current_time)

9:22:50


## 1.5 An “Aha!” insight

Thông thường có một cái nhìn insight hơn trong bài toán thì có thể giúp việc lập trình dễ hơn. Trong trường hợp này, insight của object MyTime là 3 chữ số base trên 60. Được chia làm 3 cột hours, minutes và seconds. Các quan sát này gợi ý cho một cách tiếp cận khác của cả vấn đề. Đó là convert MyTime object thành 1 cột seconds.

In [46]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):     
        
        """ Create a new MyTime object initialized to hrs, mins, secs.
           The values of mins and secs may be outside the range 0-59,
           but the resulting MyTime object will be normalized.
        """
        # Calculate total seconds to represent
        
        totalsecs = hrs*3600 + mins*60 + secs
        self.hours = totalsecs // 3600
        leftoversecs = totalsecs % 3600
        self.minutes = leftoversecs //60
        self.seconds = leftoversecs % 60
        
    def __str__(self):
        return "{0}:{1}:{2}".format(self.hours, self.minutes, self.seconds)         
    
    def to_seconds(self):
        """ Return the number of seconds represented
            by this instance
        """
        return self.hours*3600 + self.minutes*60 + self.seconds

Khối code từ line 10 đến 14 để chuẩn hóa lại các giá trị thời gian.

Ở trong lập trình định hướng đối tượng, chúng ta thật sự muốn gói dữ liệu và hoạt động lại cùng nhau. Nên chúng ta sẽ muốn làm như vậy trong class MyTime. Một giải pháp tốt là viết lại khởi tạo class, điều đó có thể giúp đương đầu với những giá trị _phút_ và _giây_ nằm ngoài chuẩn hóa. Ví dụ 3 hours 80 minutes 121 seconds thì nên được chuẩn hóa về 4 hours 22 minutes 1 second

In [47]:
nomalized_time = MyTime(3, 80, 121)
print(nomalized_time)

4:22:1


In [48]:
def add_time(t1, t2):
    secs = t1.to_seconds() + t2.to_seconds()
    return MyTime(0, 0, secs)

In [49]:
current_time = MyTime(9, 14, 30)
bread_time = MyTime(3, 35, 0)
done_time = add_time(current_time, bread_time)
print(done_time)

12:49:30


## 1.6 Another example

Nếu chúng ta muốn biết thời gian t1 liệu có sau t2 hay không. Ta có thể dùng các lệnh if lần lượt cho hours, minutes, seconds. Hoặc có thể đổi hết ra seconds rồi so sánh.

In [50]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):
        """ Create a new MyTime object initialized to hrs, mins, secs.
           The values of mins and secs may be outside the range 0-59,
           but the resulting MyTime object will be normalized.
        """
        # Calculate total seconds to represent
        totalsecs = hrs*3600 + mins*60 + secs
        self.hours = totalsecs // 3600
        leftoversecs = totalsecs % 3600
        self.minutes = leftoversecs // 60
        self.seconds = leftoversecs % 60
    
    def to_seconds(self):
        """ Return the number of seconds represented
            by this instance
        """
        return self.hours*3600 + self.minutes*60 + self.seconds
    
    def after(self, time2):
        """ Return True if I am strictly greater than time2 """
        return self.to_seconds() > time2.to_seconds()

In [51]:
def add_time(t1, t2):
    secs = t1.to_seconds() + t2.to_seconds()
    return MyTime(0,0,secs)

In [52]:
current_time = MyTime(9, 14, 30)
bread_time = MyTime(3, 35, 0)
done_time = add_time(current_time, bread_time)

In [53]:
if current_time.after(done_time):
    print("The bread will be done before it starts!")

In [54]:
current_time.after(done_time) == True

False

Có thể diễn dịch câu lệnh trên là: liệu thời gian hiện tại có phải sau thời gian hoàn thành không. Trả về False

## 1.7 Operator overloading

Trong Python, vẫn có thể cùng một operator nhưng lại mang ý nghĩa khác nhau. Ví dụ `+` trong Python có nghĩa 2 thứ khác nhau với strings hay integers. Chức năng này gọi là **operator overloading**

In [5]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)
        
    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y)
    
    def __mul__(self, other):
        return self.x * other.x + self.y + other.y
    
    def __rmul__(self, other):
        return Point(other*self.x, other*self.y)

In [6]:
p1 = Point(3,4)
p2 = Point(5,7)
print(p1*p2)

26


In [7]:
print(2*p2)

(10, 14)


- Dòng 12, 13 là **dot product** bằng việc định ra method `__mul__`
- Còn dùng 15, 16 là **scalar multiplication** bằng việc định ra method `__rmul__`

Nhưng nếu cũng là scalar multiplication nhưng không tuân theo thứ tự trên. Mà là `p2*2` thì Python lại gọi lên method `__mul__` gây lỗi

In [58]:
print(p2*2)

AttributeError: 'int' object has no attribute 'x'

## 1.8 Polymorphism

Hầu hết các methods chúng ta vừa viết chỉ hoạt động trên một loại cụ thể. Nhưng nếu chúng ta muốn áp dụng nó vào nhiều loại cùng một lúc thì viết một hàm ra bên ngoài.

In [8]:
def muladd(x,y,z):
    return x*y+z
muladd(3,2,1)

7

In [9]:
print(muladd (2,p1,p2))

(11, 15)


In [10]:
print(muladd (p1, p2, 1))

27


Một hàm nhận nhiều thông số có các loại khác nhau như thế này gọi là **polymorphic**

Nếu mọi hoạt động của hàm có thể áp dụng vào type thì hàm này có thể ứng dụng vào type đó. Ví dụ hàm `front_and_back` ở dưới có các operator sau: `copy`, `reverse` và `print`. Mà operator `copy` thì dùng được cho mọi đối tượng, `print` thì dùng được cho class Point vì chúng ta đã khai báo method `__str__`. Vậy thì class này còn thiếu operator `reverse`. Nên ta sẽ thêm `reverse` method trong Point class.

In [3]:
def front_and_back(front):
    import copy
    back = copy.copy(front)
    back.reverse()
    print(str(front) + str(back))

In [1]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __str__(self):
        return "({0}, {1})".format(self.x, self.y)
    
    def reverse(self):
        (self.x , self.y) = (self.y, self.x)

In [4]:
p = Point(3, 4)
front_and_back(p)

(3, 4)(4, 3)


kết quả hàm `front_and_back` dùng được cho object của class Point vì trong class này đã có đủ các operator trong hàm `front_and_back`

## 1.9 Glossary

- **dot product**: là một phép tính trong đại số tuyến tính, khi nhân 2 Point thì yied ra được một số
- **functional programming style**: là kiểu thiết kế chương trình, mà phần lớn các hàm là pure functions
- **modifier**: là hàm làm biến đổi thông số được pass vào. Hầu hết các modifier functions là void function
- **normalized**: chuẩn hóa là dữ liệu được đưa về cùng một tỷ lệ, base trên một giới hạn nào đó
- **operator overloading**: là phần mở rộng của các operator dựng sẵn (`+`, `-`, `*`, `/`, etc..) chúng tạo ra kết quả khác nhau với các loại thông số khác nhau. Ví dụ phép `+` thì kết quả khác nhau giữa strings và integers
- **polymorphic**: một hàm có thể operate nhiều hơn một loại. Chú ý overloading thì có những hàm khác nhau cho các loại đối tượng khác nhau. Còn polymorphic là một hàm có thể dùng trên nhiều đối tượng có type khác nhau
- **pure function**: là hàm không làm thay đổi bất kì đối tượng nào được pass làm thông số
- **scalar multiplication**: là một phép tính trong đại số tuyến tính, dùng để nhân từng tọa độ của Point với một số vô hướng.